In [1]:
import psycopg2
import pandas as pd
import datacompy
import pyodbc
import useful_functions as use

In [20]:
conn = psycopg2.connect(
    host='oneview-dev-content-aurora-cluster.cluster-cwp4vd8mllvz.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='m3CUmKz0OEOA0Mej')

In [12]:
test = pd.read_csv('../../Data/MasterfileCore/PhysicianRaceEthnicity_2021-12-08.psv', sep='|')

In [13]:
len(test)

1723895

In [21]:
ov_physician_sql = '''
    SELECT MEDICAL_EDUCATION_NUMBER, RACE_ETHNICITY FROM ONEVIEW.physician
    '''
physician = pd.read_sql_query(ov_physician_sql, conn)

In [16]:
test.medical_education_number = use.fix_me(test.medical_education_number)

In [22]:
test_compare = datacompy.Compare(physician, test, join_columns='medical_education_number', df1_name = 'oneview', df2_name = 'udrive')

In [19]:
physician

,medical_education_number,race_ethnicity
0,00102000013,White
1,00102000021,White
2,00102000030,White
3,00102000048,Asian
4,00102000056,Asian
...,...,...
1386206,03845190371,White
1386207,03845190380,Other
1386208,03845190398,White
1386209,03845190401,White


In [23]:
print(test_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns     Rows
0   oneview        2  1386211
1    udrive        2  1723895

Column Summary
--------------

Number of columns in common: 2
Number of columns in oneview but not in udrive: 0
Number of columns in udrive but not in oneview: 0

Row Summary
-----------

Matched on: medical_education_number
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 1,371,563
Number of rows in oneview but not in udrive: 14,648
Number of rows in udrive but not in oneview: 352,332

Number of rows with some compared columns unequal: 75,511
Number of rows with all compared columns equal: 1,296,052

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 75,511

Columns with Unequal Values or Types
-------------------------------

In [5]:
physician.groupby('race_ethnicity').count()

,medical_education_number
race_ethnicity,
American Indian or Alaskan Native,2255
Asian,140565
Black or African American,39973
"Hispanic, Latino, or Spanish Origin",53648
Mixed Race/Ethnicity,224409
Native Hawaiian or Other Pacific Islander,506
Other,23629
Tribal Affiliation,18
Unknown,249871


In [ ]:
ov_program_sql = '''
    SELECT * FROM ONEVIEW.RESIDENCY_PROGRAM
    '''
ov_program = pd.read_sql_query(ov_program_sql, conn)

In [ ]:
ov_personnel_sql = '''
    SELECT * FROM ONEVIEW.residency_program_personnel_member
    '''
ov_personnel = pd.read_sql_query(ov_personnel_sql, conn)

In [ ]:
ov_physician_sql = '''
    SELECT * FROM ONEVIEW.residency_program_physician
    '''
ov_physician = pd.read_sql_query(ov_physician_sql, conn)

In [ ]:
ov_institution_sql = '''
    SELECT * FROM ONEVIEW.residency_program_institution
    '''
ov_institution = pd.read_sql_query(ov_institution_sql, conn)

In [ ]:
program = pd.read_csv('../../Data/MasterfileCore/ProgramInformation.txt', sep='|', error_bad_lines=False, encoding='latin')
program = program[program.pgm_activity_code==0]

In [ ]:
ov_personnel

In [ ]:
program_columns = {
    'pgm_id':'id',
    'pgm_spec':'specialty',
    'pgm_inst_control_type':'institution_control',
    'pgm_sequence_number':'sequence_number',
    'pgm_federal_code':'federal_code',
    'pgm_region_code':'region_code',    
    'prm_acgme_accred_ind':'acgme_accredited',
    'pgm_name':'name',
    'pgm_web_address':'web_address',
    'pgm_contact_dir_ind':'contact_director',
    'pgm_accred_status':'accreditation_status',
    'pgm_accred_eff_dt':'accreditation_effective_date',
    'pgm_init_accred_dt':'initial_accreditation_date', 
    'pgm_accred_length':'accreditation_length',
    'pgm_yrs_offered':'duration',
    'pgm_gov_affil_ind':'govenrmnet_affiliated',
    'pgm_gme_eq_yr':'graduate_medical_education_equivalent_years',
    'pgm_prelim_yrs_req_ind':'preliminary_years_required',
    'pgm_prelim_pos_offered':'preliminary_positions_offered',
    'pgm_type':'type',
    'pgm_chg_size':'max_residents_accepted_increase',
    'pgm_%@_prim_site':'percent_at_primary_site',
    'pgm_prim_site':'primary_site',
    'pgm_core_pgm_id':'core_program',
    'official_addr_ind':'official_address',
    'pgm_sf_match':'uses_sf_match',
    'pgm_oth_match_ind':'other_match_indicator',
    'pgm_oth_match':'other_match',
    'pgm_addit_educ_accred_length':'additional_education_accreditation_length',
    'last_upd_dt':'last_update_date',
    'last_upd_type':'last_update_type',
    'AOA_Ind':'american_osteopathic_association_indicator', 
    'AOA_Program_ID':'american_osteopathic_association_indicator_program_id',
    'Osteopathic_principles':'osteopathic_principles'
    
}

In [ ]:
program = program.rename(columns=program_columns)

In [ ]:
ov_institution

In [ ]:
ov_program

In [ ]:
program_compare = datacompy.Compare(ov_program, yes, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [ ]:
print(program_compare.report())

In [ ]:
inst = pd.read_csv('../../Data/MasterfileCore/ProgramInstitution.txt', sep='|', encoding='latin')
insti = inst[inst.affiliation_type=='S']

In [ ]:
program = pd.merge(program, insti, left_on='id', right_on='pgm_id')

In [ ]:
yes = program[program.ins_id.isin(ov_institution.id)]

In [ ]:
institution = pd.read_csv('../../Data/MasterfileCore/InstitutionInformation.txt', sep='|', error_bad_lines=False)

In [ ]:
addresses = pd.read_csv('../../Data/MasterfileCore/ProgramAddress.txt', sep='|', error_bad_lines=False, encoding='latin')

In [ ]:
personnel = pd.read_csv('../../Data/MasterfileCore/ProgramPersonnel.txt', sep='|', encoding='latin', error_bad_lines=False)

In [ ]:
personnel = personnel.rename(columns={'pers_name_sfx':'suffix_name', 'pers_name_last':'last_name',
       'pers_name_first':'first_name', 'pers_name_mid':'middle_name', 'pers_deg1':'degree_1', 'pers_deg2':'degree_2',
       'pers_deg3':'degree_3', 'last_upd_dt':'last_update_date'})

In [ ]:
personnel['program']=personnel.pgm_id

In [ ]:
personnel = personnel[personnel.pers_type=='D']

In [ ]:
personnel['id'] = personnel['pgm_id'].astype(str) + personnel['aamc_id'].astype(str)

In [ ]:
def fix_id(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    if len(num) == 9:
        num = '0' + num
    return num
insti['pgm_id']=[fix_id(x) for x in insti.pgm_id]

In [ ]:
def fix_aamc_id(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    while len(num) < 8:
        num = '0' + num
    return num
personnel['aamc_id']=[fix_aamc_id(x) for x in personnel.aamc_id]

In [ ]:
test = personnel[personnel.program.isin(program.id)]

In [ ]:
personnel_compare = datacompy.Compare(ov_personnel, test, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [ ]:
print(personnel_compare.report())

In [ ]:
ov_personnel.id

In [ ]:
def fix_personnel_id(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    while len(num) < 18:
        num =  num + '0'
    return num
ov_personnel['id'] = [fix_personnel_id(x) for x in ov_personnel.id]

In [ ]:
resident_link = pd.read_csv('../../Data/MasterfileCore/RESIDENT_LINK.csv')

In [ ]:
resident_link['program']=[fix_id(x) for x in resident_link.program]

In [ ]:
resident_link['medical_education_number'] = use.fix_me(resident_link.medical_education_number)

In [ ]:
pd.merge(resident_link, ov_physician, on = list(resident_link.columns), how='left')

In [ ]:
institution

In [ ]:
def fix_id_2(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    if len(num) == 5:
        num = '0' + num
    return num
insti['id'] = [fix_id_2(x) for x in insti.ins_id]
institution['id'] = [fix_id_2(x) for x in institution.ins_id]
meri_institution = pd.merge(insti, institution, on='id')

In [ ]:
meri_institution = meri_institution.rename(columns={'affiliation_type':'affiliation','ins_name':'name','last_upd_dt_y':'last_update_date'})

In [ ]:
meri_institution

In [ ]:
inst_compare = datacompy.Compare(ov_institution, institution[institution.ins_affiliation_type=='S'], join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [ ]:
print(inst_compare.report())

In [ ]:
inst_compare.df1_unq_rows

In [ ]:
meri_institution[meri_institution.name=='North Alabama Medical Center']

In [ ]:
institution['affiliation']=institution.ins_affiliation_type
institution['last_update_date']=institution.last_upd_dt

In [ ]:
ov_institution.groupby('affiliation').count()

In [ ]:
institution['name'] = institution.ins_name

In [ ]:
institution[institution.ins_affiliation_type=='S']

In [ ]:
ov_institution

In [ ]:
ov_program

In [ ]:
inst

In [ ]:
addresses['pgm_id']=[fix_id(x) for x in addresses.pgm_id]
address = addresses[addresses.addr_type=='D']

In [ ]:
program = pd.merge(address, program, left_on='pgm_id', right_on='id')

In [ ]:
ov_program.columns

In [ ]:
inst['ins_id'] = [fix_id_2(x) for x in inst.ins_id]

In [ ]:
insti['ins_id'] = [fix_id_2(x) for x in insti.ins_id]

In [ ]:
institution[institution.ins_affiliation_type=='S']

In [ ]:
ov_program

In [ ]:
program.columns

In [ ]:
ov_program

In [ ]:
program_2 = pd.merge(program, insti[insti.ins_id.isin(institution.ins_id)], left_on='id', right_on='program')

In [ ]:
insti['program']=[fix_id(x) for x in insti.pgm_id]

In [ ]:
fix_id(200121109)

In [ ]:
program_compare = datacompy.Compare(ov_program, program_2, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [ ]:
program_2 = program_2.rename(columns = {'id_x':'id'})

In [ ]:
print(program_compare.report())

In [ ]:
program[program.pgm_id=='1201611099']

In [ ]:
program

In [ ]:
ov_personnel

In [ ]:
personnel[personnel.pgm_id.isin(ov_program.id)]

In [ ]:
personnel_compare_2 = datacompy.Compare(ov_personnel, personnel, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [ ]:
print(personnel_compare_2.report())

In [ ]:
personnel_compare_2.df1_unq_rows.to_csv('../../Data/OneView_Testing/Personnel_ID_Examples.csv', index=False)

In [ ]:
print(program_compare.report())

In [ ]:
inst_compare.df1_unq_rows.sample().to_csv('../../Data/OneView_Testing/Institution_Weird_Row.csv', index=False)

In [ ]:
program_compare.df2_unq_rows[ov_program.columns].to_csv('../../Data/OneView_Testing/Residency_Program_Missing_Rows.csv', index=False)

In [ ]:
program_2 = program_2.rename(columns={'govenrmnet_affiliated':'government_affiliated','id_y':'institution', 'uses_sf_match':'uses_sfmatch','pri_clinical_loc_ind':'primary_clinical_location','addr1':'address_1','addr2':'address_2','addr3':'address_3','american_osteopathic_association_indicator_program':'american_osteopathic_association_program', 'zip+4':'zipcode'})

In [ ]:
['government_affiliated', 'institution', 'uses_sfmatch', 'primary_clinical_location', 'address_3', 'american_osteopathic_association_program', 'address_2', 'address_1', 'zipcode']

In [ ]:
program_2.columns

In [ ]:
program_compare.sample_mismatch(column="american_osteopathic_association_program").dropna()